## Run train and test
* famafrench生成残差时间序列后输入到模型训练中
* 用于生成绘制cumulative_returns,turnovers,short

In [6]:
import numpy as np
import pandas as pd
import os
import yaml
import sys
sys.path.append(r'E:\dlsa')

from train_test import test,train
from preprocess import preprocess_ou,preprocess_fourier,preprocess_cumsum
from models.OUFFN import OUFFN
from models.CNNTransformer import CNNTransformer
from models.FourierFFN import FourierFFN

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, DayLocator ,MonthLocator, YearLocator

In [7]:
current_directory = os.getcwd()
base = r'E:\dlsa'
random_seed_list = [42]
#模型参数设置
ou_obj = OUFFN(logdir=current_directory,
                lookback=30,
                random_seed=42, 
                device="cpu",
                hidden_units=[4, 4, 4, 4], 
                dropout=0.01)
fft_obj = FourierFFN(logdir=current_directory,
                lookback=30,
                random_seed=42, 
                device="cpu",
                hidden_units=[30, 16, 8, 4], 
                dropout=0.01)
cnn_obj = CNNTransformer(logdir=current_directory,
                random_seed=42, 
                lookback=30,
                device="opencl",
                normalization_conv=True, 
                filter_numbers=[1, 8], 
                attention_heads=4, 
                use_convolution=True,
                hidden_units=2*8, 
                hidden_units_factor=2,
                dropout=0.01, 
                filter_size=2, 
                use_transformer=True)

In [8]:
#读取数据
# excess_returns = pd.read_csv(base+"/Excess_Returns_adjusted.csv",index_col=('Date'))
# excess_returns.index = pd.to_datetime(excess_returns.index,format="%Y%m%d")
# excess_returns

,AAPL,AFL,FL,LOW,NKE,PGR,RRC,SEE,VMC,WMT
Date,,,,,,,,,,
1982-01-05,-0.051533,-0.016536,0.020148,-0.000400,-0.026947,-0.054246,-0.071828,-0.000400,0.001904,-0.029810
1982-01-06,-0.012376,-0.000400,-0.000400,-0.029527,-0.018580,-0.032920,-0.000400,-0.000400,-0.002699,-0.039797
1982-01-07,-0.079194,-0.000400,-0.007111,0.029601,0.008858,-0.000400,-0.000400,0.004145,-0.005008,0.002759
1982-01-08,0.045662,0.016000,0.006357,-0.029527,0.045469,-0.008799,-0.000400,0.008649,-0.000400,0.024756
1982-01-11,-0.063300,-0.016536,-0.007111,-0.000400,0.008372,-0.000400,-0.077323,-0.031790,-0.000400,-0.015741
...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.007712,0.007122,-0.015457,0.002093,-0.001129,0.008204,-0.044837,-0.003028,0.003574,0.001955
2020-12-28,0.035765,0.001825,0.022031,-0.004485,0.005862,-0.001933,-0.075391,-0.012365,0.020597,0.011986
2020-12-29,-0.013315,-0.003870,-0.000743,-0.009257,-0.006038,-0.004382,-0.040000,-0.005491,-0.023260,-0.006335


In [9]:
#定义基本配置
epoch_num = 250
l = [3] #因子模型列表
model_tags = ['OUFFN', 'FourierFFN', 'CNNTransformer']
configs_list = ['ouffn-full.yaml', 'fourierffn-full.yaml', 'cnntransformer-full.yaml']
preprocess_list = [preprocess_ou, preprocess_fourier, preprocess_cumsum]
model_list = [ou_obj, fft_obj, cnn_obj]
model_list = [cnn_obj]
initialOOSyear=2002

In [11]:
#循环进行训练和测试
for idx, preprocess1 in enumerate(preprocess_list): #遍历每种预处理方法
    print(f"starting {preprocess1}")
    model_tag1 = model_tags[idx]
    config1 = configs_list[idx]
    model1 = model_list[idx]
    
    with open(fr'{base}/configs/{config1}', 'r') as yaml_file: #读取配置信息
        config = yaml.safe_load(yaml_file)

    for obj1 in objectives:
        print(f"starting{obj1}")
        for _ in l:
            print(f"starting factor {_} model")
            #导入残差数据
            data = pd.read_csv(base+f"/residuals/_{_}_factors_{initialOOSyear}_initialOOSyear.csv", index_col=('Date'))
            data.index = pd.to_datetime(data.index,format="%Y%m%d")
            factors_num = _ #使用的因子数量
            #考虑交易成本的模型测试
            vari = f'{obj1}_WithCost_{factors_num}_factor'
            vzl_test = test(Data=np.array(data), 
                            daily_dates=list(data.index),
                            model=model1,
                            preprocess=preprocess1,
                            config=config,
#                             excess_returns_data=np.array(excess_returns),
                            num_epochs=epoch_num,
                            device='cpu',
                            model_tag=model_tag1,
                            batchsize=100,
                            trans_cost=0.0005,
                            hold_cost=0.0001,
                            vari=vari,
                            objective=obj1)
            # 不考虑交易成本的模型测试                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
            vari = f'{obj1}_WithNoCost_{factors_num}_factor'
            vzl_test = test(Data=np.array(data), 
                            daily_dates=list(data.index),
                            model=model1,
                            preprocess=preprocess1,
                            config=config,
#                             excess_returns_data=np.array(excess_returns),
                            num_epochs=epoch_num,
                            device='cpu',
                            model_tag=model_tag1,
                            batchsize=100,
                            trans_cost=0.000,
                            hold_cost=0.000,
                            vari=vari,
                            objective=obj1)

starting <function preprocess_cumsum at 0x0000024E53288360>
startingmeanvar
starting factor 3 model
running test in 1/16 this many times test will play
1/16 the value of t started
running epoch 0/250
running epoch 20/250
running epoch 40/250
running epoch 60/250
running epoch 80/250
running epoch 100/250
running epoch 120/250
running epoch 140/250


KeyboardInterrupt: 